# Variational Quantum Error Correction with QVECTOR

In this notebook, we will optimize a Quantum Error Correction (QEC) problem. 
Here, our set of states $\{\psi_i\}_{i=1}^N$ are sampled from the Haar distribution (or an approximation thereof) and we optimize the cost 
$$L(\vec{p}, \vec{q}) = \sum_{\vert \psi_i \rangle \in \mathcal{S}} \frac{1}{\vert\mathcal{S}\vert} E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q})$$
where $\vec{p}, \vec{q}$ are parameters of a variational circuit, and $E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q}) = \langle \psi_i \vert \mathcal{V}(\vec{p})^\dagger \mathcal{W}(\vec{q}) \mathcal{N} \mathcal{V}(\vec{p}) \vert \psi_i \rangle$, with $\mathcal{V}$ the encoding operator, $\mathcal{N}$ the noise model, and $\mathcal{W}$ the decoding operator. The noise model here will be a bitflip channel (with probability $0.1$) acting on all qubits.

In [1]:
import typing as ty
import pennylane as qml
from pennylane import numpy as np
from refoqus import Refoqus

# Initialise the number of qubits that interest us.
k, n, r = 1, 3, 0
nbqbits = n + r
# Number of layers to use in the V and W parametrised circuits.
layers = 2
# Parametrised template that will be used to construct the V and W circuits.
ansatz = qml.BasicEntanglerLayers
#ansatz = qml.StronglyEntanglingLayers

The dataset is composed of random unitary matrices sampled from a 2-design (or approximate 2-design here). For 1 qubit, the dataset can be simplified to be only the 6 one-qubit stabiliser states.

In [2]:
def random_angles(*shape: int) -> np.ndarray:
    return 2 * np.pi * np.random.rand(*shape)

if k == 1:
    # Only consider the 6 one qubit stabilizer states:
    dataset_size = 6
    dataset = [
        [qml.Identity(wires=[0])], [qml.PauliX(wires=[0])],  # |0>, |1>
        [qml.Hadamard(wires=[0])], [qml.PauliX(wires=[0]), qml.Hadamard(wires=[0])],  # |+>, |->
        [qml.Hadamard(wires=[0]), qml.S(wires=[0])], [qml.PauliX(wires=[0]), qml.Hadamard(wires=[0]), qml.S(wires=[0])]  # |+i>, |-i>
    ]
    
elif k > 1:
    # Build the dataset here with qml.SimplifiedTwoDesign
    # See https://docs.pennylane.ai/en/stable/code/api/pennylane.SimplifiedTwoDesign.html
    dataset_size = 200
    dataset = [[qml.SimplifiedTwoDesign(initial_layer_weights=random_angles(k), weights=random_angles(layers, k - 1, 2), wires=range(k))] for _ in range(dataset_size)]

The cost function $E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q})$ can be computed by applying $ S^\dagger \mathcal{V}(\vec{p})^\dagger \mathcal{W}(\vec{q}) \mathcal{N} \mathcal{V}(\vec{p}) S$ to the $\vert 0 \rangle$ and measure the probability of getting the full zero outcome $\vert 0 \rangle^{\otimes n}$. 

To avoid using a global cost function that would inevitably suffer from the Barren Plateau phenomenon, we are using the local cost function defined by the Hamiltonian
$$
H = \mathbb{I} - \frac{1}{k} \sum_{i=0}^{k-1} \mathbb{I}_{\bar{i}} \otimes \vert 0 \rangle \langle 0 \vert_i
$$
where $\mathbb{I}_{\bar{i}} \otimes \vert 0 \rangle \langle 0 \vert_i$ means that we are projecting only the $i$-th qubit to the $\vert 0 \rangle$ state, leaving the other qubits.

Note that our goal is to recover pure states, for which the $\mathbb{I}$ part will evaluate to $1$, so we do not have to include it in the measured Hamiltonian and we can add it during post-processing.

In [3]:
# Get the Hamiltonian of interest
coefficients_cost = [- 1.0 / k for _ in range(k)]
projector = np.zeros((2, 2))
projector[0, 0] = 1
vqec_hamiltonian_term = [qml.Hermitian(projector, wires=i) for i in range(k)]
hamiltonian_of_interest = qml.Hamiltonian(coefficients_cost, vqec_hamiltonian_term)

Next, we define functions to evaluate the true cost during optimization.

In [4]:
def circuit_construction(
    weights: np.ndarray,
    data_circuit: ty.List[qml.operation.Operation],
    parameterised_circuit = None,
    noise_circuit = None,
):
    if parameterised_circuit is None:
        parameterised_circuit = ansatz
    if noise_circuit is None:
        def noise_circuit():
            for i in range(n):
                qml.BitFlip(0.1, wires=i)

    encoding_shape = parameterised_circuit.shape(layers, n)
    decoding_shape = parameterised_circuit.shape(layers, n + r)
    encoding_size = np.prod(encoding_shape)
    decoding_size = np.prod(decoding_shape)
    encoding_weights = weights[:encoding_size].reshape(encoding_shape)
    decoding_weights = weights[encoding_size:encoding_size + decoding_size].reshape(decoding_shape)
    
    # Apply S         to range(k)     to prepare an approximate 2-design state
    for op in data_circuit:
        qml.apply(op)
    
    # Apply V(p)      to range(n)     to encode the state
    parameterised_circuit(encoding_weights, wires=range(n))
    # Apply potential noise
    noise_circuit()
    # Apply W(q)      to range(n + r) to correct potential errors
    parameterised_circuit(decoding_weights, wires=range(n + r))
    # Apply V^{-1}(p) to range(n)     to decode the state
    qml.adjoint(parameterised_circuit(encoding_weights, wires=range(n)))
    
    # Apply S^{-1}    to range(k)     to un-prepare.
    for op in reversed(data_circuit):
        qml.apply(qml.adjoint(op))
    
def cost_function(
    weights: np.ndarray,
    hamiltonian_terms: qml.operation.Operator,
    data_circuit: ty.List[qml.operation.Operation],
    parameterised_circuit = None,
):
    # Here noise_circuit is left intentionally to default, allowing the default BitFlip noise
    # to be applied.
    circuit_construction(weights, data_circuit, parameterised_circuit)
    return qml.sample(hamiltonian_terms)

In order to have access to the non-noisy cost during the optimisation process we define functions to estimate it using perfect, noise-free simulators.

In [5]:
analytic_dev = qml.device("default.mixed", wires=nbqbits, shots=None)

@qml.qnode(analytic_dev)
def cost_analytic_one_circuit(weights, index_datapoint, parameterised_circuit = None):
    # Here, noise_circuit is explicitely set to a lambda that does nothing.
    # This is to prevent any noise from being added to the computation because we want a noise-free estimation in this routine.
    circuit_construction(weights, dataset[index_datapoint], parameterised_circuit, noise_circuit=lambda : None)
    return qml.expval(hamiltonian_of_interest)

def cost_analytic_alldataset(weights, parameterised_circuit = None):
    cost = 0.0
    for m in range(dataset_size):
        cost += cost_analytic_one_circuit(weights, m, parameterised_circuit)
    cost = 1.0 + cost / dataset_size
    return cost

Now, the ansatz is defined as with StronglyEntanglingLayers. We also sample initial values and the corresponding cost.

Our adaptative optimizer will be Refoqus where we provide the necessary arguments as follows and we perform niter iterations.

In [6]:
parameter_size: int = np.prod(ansatz.shape(layers, n)) + np.prod(ansatz.shape(layers, n + r))
params = random_angles(parameter_size)

opt = Refoqus(nbqbits, dataset, vqec_hamiltonian_term, coefficients_cost, param_shape=(parameter_size,), function_cost_term_tosample=cost_function, min_shots=2, device_name="default.mixed")
niter = 20

cost_refoqus = [cost_analytic_alldataset(params, ansatz)]
shots_refoqus = [0]

for i in range(niter):
    params = opt.step(params)
    cost_refoqus.append(cost_analytic_alldataset(params, ansatz))
    shots_refoqus.append(opt.shots_used)
    print(f"Step {i}: cost = {cost_refoqus[-1]}, shots_used = {shots_refoqus[-1]}")

Step 0: cost = 0.7885592318715495, shots_used = 48
Step 1: cost = 0.6046781891568378, shots_used = 108
Step 2: cost = 0.33305975485748573, shots_used = 162
Step 3: cost = 0.12745506986684318, shots_used = 288
Step 4: cost = 0.016385763262979025, shots_used = 416
Step 5: cost = 0.006900780477326118, shots_used = 704
Step 6: cost = 0.04513347218667174, shots_used = 1068
Step 7: cost = 0.00441184534694683, shots_used = 1628
Step 8: cost = 0.02682783170616554, shots_used = 2254
Step 9: cost = 0.031992724753818536, shots_used = 2710
Step 10: cost = 0.08351446141118402, shots_used = 3020
Step 11: cost = 0.08187249083468018, shots_used = 3312
Step 12: cost = 0.05533068625900561, shots_used = 3604
Step 13: cost = 0.111717896296195, shots_used = 3764
Step 14: cost = 0.021075139974568446, shots_used = 3988
Step 15: cost = 0.020913140362164184, shots_used = 4262
Step 16: cost = 0.03615837859137494, shots_used = 4572
Step 17: cost = 0.027974805502202993, shots_used = 4844
Step 18: cost = 0.0167910